In [101]:
import numpy as np
import pandas as pd
import requests
import config
import os
import json
from pandas.io.json import json_normalize
import pygsheets

##Schedule_a API guide: https://api.open.fec.gov/developers/#/receipts/get_schedules_schedule_a_

In [102]:
### MULTI-CANDIDATE SEARCH ###
#Set search for all 2020 (two-year transaction period) Maine Congressional candidates
cand_state = 'ME'
period='2020'
parameters = {'election_year':period
            ,'state':cand_state
            ,'api_key':config.api_key}

#Requests candidate info
r_cands = requests.get('https://api.open.fec.gov/v1/candidates/search',params=parameters).json()

#Locates and sets Committee ID from 'principal_committees' sub-array
#Output list of IDs
comm_ids=json_normalize(data=r_cands['results'],record_path='principal_committees')['committee_id'].tolist()
comm_ids

['C00710087',
 'C00314575',
 'C00709899',
 'C00653816',
 'C00703413',
 'C00706739',
 'C00683193',
 'C00433391',
 'C00709279',
 'C00712323',
 'C00694364']

In [103]:
#Print committee list to validate
cand_list=json_normalize(data=r_cands['results'],record_path='principal_committees')[['committee_id','name','party']]
print(cand_list)
cand_list.iloc[id]['name']

   committee_id                             name party
0     C00710087         MICHAEL BUNKER FOR MAINE   DEM
1     C00314575              COLLINS FOR SENATOR   REP
2     C00709899            SARA GIDEON FOR MAINE   DEM
3     C00653816  COMMITTEE TO ELECT JARED GOLDEN   DEM
4     C00703413            FRIENDS OF BRE KIDMAN   DEM
5     C00706739            DEREK FOR U.S. SENATE   REP
6     C00683193      DR CAT LONDON FOR US SENATE   DEM
7     C00433391             PINGREE FOR CONGRESS   DEM
8     C00709279                 SWEET FOR SENATE   DEM
9     C00712323       JON TREACY FORWARD 4 MAINE   DEM
10    C00694364   DANIELLE VANHELSING FOR SENATE   IND


'DANIELLE VANHELSING FOR SENATE'

In [104]:
#Initialize dataframe collector for itemized contribs
dfs=[]
id=int(0)
p=int(1)
    
#Loop through pages
for x in range(len(comm_ids)-1):
    
    #Print out candidate
    print(cand_list.iloc[id]['name'])
    
    querydict = {'per_page':'100'
                ,'api_key':config.api_key
                ,'committee_id':comm_ids[id]
                ,'page':p
                ,'sort_nulls_last':'false'
                ,'sort':'-contribution_receipt_date'
                ,'sort_hide_null':'false'
                }
    
    r = requests.get('https://api.open.fec.gov/v1/schedules/schedule_a/efile/',params=querydict).json()
    
    #Loop through pages
    while p <= r['pagination']['pages']:
        
        print('page: '+str(p)+','+str(r['pagination']['per_page'])+' records')
        
        querydict.update(page=p)
        #Pull new results
        r = requests.get('https://api.open.fec.gov/v1/schedules/schedule_a/efile/',params=querydict).json()
        
        #Add results to dataframe
        df = json_normalize(r['results'])
        dfs.append(df)
        p=p+1
    
    #Reset to page 1
    p=int(1)
    #Increment to next candidate
    id=id+1

print('Job Complete')

MICHAEL BUNKER FOR MAINE
COLLINS FOR SENATOR
page: 1,100 records
page: 2,100 records
page: 3,100 records
page: 4,100 records
page: 5,100 records
page: 6,100 records
page: 7,100 records
page: 8,100 records
page: 9,100 records
page: 10,100 records
page: 11,100 records
page: 12,100 records
page: 13,100 records
page: 14,100 records
page: 15,100 records
page: 16,100 records
page: 17,100 records
page: 18,100 records
page: 19,100 records
page: 20,100 records
page: 21,100 records
page: 22,100 records
page: 23,100 records
page: 24,100 records
page: 25,100 records
page: 26,100 records
page: 27,100 records
page: 28,100 records
page: 29,100 records
page: 30,100 records
page: 31,100 records
page: 32,100 records
page: 33,100 records
page: 34,100 records
page: 35,100 records
page: 36,100 records
page: 37,100 records
page: 38,100 records
page: 39,100 records
page: 40,100 records
page: 41,100 records
page: 42,100 records
page: 43,100 records
page: 44,100 records
SARA GIDEON FOR MAINE
page: 1,100 record

page: 2,100 records
page: 3,100 records
page: 4,100 records
page: 5,100 records
page: 6,100 records
page: 7,100 records
page: 8,100 records
page: 9,100 records
page: 10,100 records
page: 11,100 records
page: 12,100 records
page: 13,100 records
page: 14,100 records
page: 15,100 records
page: 16,100 records
page: 17,100 records
page: 18,100 records
page: 19,100 records
page: 20,100 records
page: 21,100 records
page: 22,100 records
page: 23,100 records
page: 24,100 records
page: 25,100 records
page: 26,100 records
page: 27,100 records
page: 28,100 records
page: 29,100 records
page: 30,100 records
page: 31,100 records
page: 32,100 records
page: 33,100 records
SWEET FOR SENATE
page: 1,100 records
page: 2,100 records
JON TREACY FORWARD 4 MAINE
Job Complete


In [97]:
itemdf=pd.concat(dfs,sort=False)
itemdf=itemdf.drop_duplicates(subset='transaction_id')
itemdf

,amendment_indicator,back_reference_schedule_name,back_reference_transaction_id,committee.candidate_ids,committee.city,committee.committee_id,committee.committee_type,committee.committee_type_full,committee.cycle,committee.cycles,...,image_number,line_number,load_timestamp,memo_code,memo_text,pdf_url,pgo,related_line_number,report_type,transaction_id
0,A,SA12,AFF8DC83E57D6456D9F3,[S6ME00159],BANGOR,C00314575,S,Senate,2020,"[1996, 1998, 2000, 2002, 2004, 2006, 2008, 201...",...,201907159151155255,12,2019-07-15T21:14:42+00:00,X,None,http://docquery.fec.gov/cgi-bin/fecimg/?201907...,P2020,1232,F3N,A60F95D3FAE2B4FD0909
1,A,SA12,AFF8DC83E57D6456D9F3,[S6ME00159],BANGOR,C00314575,S,Senate,2020,"[1996, 1998, 2000, 2002, 2004, 2006, 2008, 201...",...,201907159151155259,12,2019-07-15T21:14:42+00:00,X,None,http://docquery.fec.gov/cgi-bin/fecimg/?201907...,P2020,1243,F3N,A7563EB6CEE424EE6B90
2,A,SA12,AFF8DC83E57D6456D9F3,[S6ME00159],BANGOR,C00314575,S,Senate,2020,"[1996, 1998, 2000, 2002, 2004, 2006, 2008, 201...",...,201907159151155252,12,2019-07-15T21:14:41+00:00,X,None,http://docquery.fec.gov/cgi-bin/fecimg/?201907...,P2020,1221,F3N,A9273561CA7524445A8C
3,A,SA12,AFF8DC83E57D6456D9F3,[S6ME00159],BANGOR,C00314575,S,Senate,2020,"[1996, 1998, 2000, 2002, 2004, 2006, 2008, 201...",...,201907159151155255,12,2019-07-15T21:14:42+00:00,X,None,http://docquery.fec.gov/cgi-bin/fecimg/?201907...,P2020,1231,F3N,A9CE5AABBC81C441A827
4,A,SA12,AFF8DC83E57D6456D9F3,[S6ME00159],BANGOR,C00314575,S,Senate,2020,"[1996, 1998, 2000, 2002, 2004, 2006, 2008, 201...",...,201907159151155257,12,2019-07-15T21:14:42+00:00,X,None,http://docquery.fec.gov/cgi-bin/fecimg/?201907...,P2020,1238,F3N,A5376D998DD9D445F8EE
5,A,SA12,AFF8DC83E57D6456D9F3,[S6ME00159],BANGOR,C00314575,S,Senate,2020,"[1996, 1998, 2000, 2002, 2004, 2006, 2008, 201...",...,201907159151155259,12,2019-07-15T21:14:42+00:00,X,None,http://docquery.fec.gov/cgi-bin/fecimg/?201907...,P2020,1242,F3N,AD0F4D9FD7A6A461E839
6,A,SA12,A4438B0E72AD841208BE,[S6ME00159],BANGOR,C00314575,S,Senate,2020,"[1996, 1998, 2000, 2002, 2004, 2006, 2008, 201...",...,201907159151155249,12,2019-07-15T21:14:41+00:00,X,None,http://docquery.fec.gov/cgi-bin/fecimg/?201907...,P2020,1213,F3N,AB086963DB6C2407AAA0
7,A,SA12,AFF8DC83E57D6456D9F3,[S6ME00159],BANGOR,C00314575,S,Senate,2020,"[1996, 1998, 2000, 2002, 2004, 2006, 2008, 201...",...,201907159151155251,12,2019-07-15T21:14:41+00:00,X,None,http://docquery.fec.gov/cgi-bin/fecimg/?201907...,P2020,1220,F3N,A833561F6709D4AF0906
8,A,SA12,AFF8DC83E57D6456D9F3,[S6ME00159],BANGOR,C00314575,S,Senate,2020,"[1996, 1998, 2000, 2002, 2004, 2006, 2008, 201...",...,201907159151155253,12,2019-07-15T21:14:42+00:00,X,None,http://docquery.fec.gov/cgi-bin/fecimg/?201907...,P2020,1225,F3N,A62DBED096B1E4786B68
9,A,SA12,AFF8DC83E57D6456D9F3,[S6ME00159],BANGOR,C00314575,S,Senate,2020,"[1996, 1998, 2000, 2002, 2004, 2006, 2008, 201...",...,201907159151155255,12,2019-07-15T21:14:42+00:00,X,None,http://docquery.fec.gov/cgi-bin/fecimg/?201907...,P2020,1230,F3N,A88B56F52279840EEB07


In [99]:
#Itemized dataframe query
itemdf=itemdf[[
                'committee.city'
                ,'committee.committee_id'
                ,'committee.committee_type_full'
                ,'committee.cycle'
                ,'committee.name'
                ,'committee.party_full'
                ,'committee.state_full'
                ,'contribution_receipt_amount'
                ,'contribution_receipt_date'
                ,'contributor_aggregate_ytd'
                ,'contributor_city'
                ,'contributor_employer'
                ,'contributor_first_name'
                ,'contributor_last_name'
                ,'contributor_name'
                ,'cycle'
                ,'contributor_occupation'
                ,'contributor_state'
                ,'contributor_zip'
                ,'entity_type'
                ,'filing.coverage_end_date'
                ,'filing.coverage_start_date'
                ,'filing.filed_date'
                ,'filing.is_amended'
                ,'fec_election_type_desc'
                ,'memo_text'
                ,'filing.pdf_url'
                ,'transaction_id']]
len(itemdf.index)

25812

In [100]:
#Write itemized individual results to local CSV
cwd = os.getcwd()
itemdf.to_csv(cwd+'/maine-fed-raw-itemized-receipts.csv')

In [92]:
#Write itemized individual raw results to Google Sheet
cwd = os.getcwd()

#Google Credentials
gc = pygsheets.authorize(service_file=cwd+'/me-congress-2020-creds.json')

#Select sheet and worksheet
sh = gc.open('me-congress-2020')
#sh = gc.open_by_key('1AKrgHT9NLpoddV16B7_M_0PEjJmMQAGtXJUnLCTDHjA')
wks = sh[3]

#Clear sheet before load
wks.clear(start='A1',fields='*')

#Write contribs dataframe to sheet
wks.set_dataframe(itemdf,(1,1))

HttpError: <HttpError 400 when requesting https://sheets.googleapis.com/v4/spreadsheets/12bHf1qEtKtGGje0a3lBJJ8r-IvYcs3yl5jxbP6jNMCo/values/Raw%20Itemized%20filings%21A7405%3AAB9256?valueInputOption=USER_ENTERED&alt=json returned "Range ('Raw Itemized filings'!A7405:AB9256) exceeds grid limits. Max rows: 7404, max columns: 84">